In [1]:
from IPython.display import clear_output
!pip install requests
!pip install --upgrade pandas
!pip install konlpy
!pip install gensim
!cp -R /home/work/deep/jdk1.8.0_201 /usr/lib/jvm
clear_output()

In [3]:
!pip install plotly
import pandas as pd
import numpy as np
import requests
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('kyoh', 'xLxYyOTECJ48ofwzrn2j')
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
clear_output()

In [ ]:
### 1) 국제 원유가격 데이터에서 예측할 두바이유 데이터 로드 및 추세 확인
### 2) 피쳐로 사용할 데이터 탐색 
### - 국제 유가 관련 뉴스 데이터
### - 다우 존스 데이터
### - 환율 데이터
### - 코스피 데이터
### - Brent 데이터
### - WTI 데이터
### - 국제 유가에 주가 상승 영향을 받는 국내 주식 종목
###   (SK이노베이션,S-OIL,금호석유)
### - 국제 유가에 주가 하락 영향을 받는 국내 주식 종목
###   (아시아나항공,대한항공,현대상선)

In [ ]:
pd.DataFrame(data={'time':timestamp,'SK이노베이션':data})

#### 국제 원유가격(Brent, WTI, 두바이) 데이터 로드 및 전처리

In [22]:
df_oil_price = pd.read_csv('국제_원유가격20080607_20190610.csv', encoding='UTF-8')
df_oil_price.columns = ['time','dubai','brent','WIT']

df_oil_price['time'] = '20'+df_oil_price.time
df_oil_price['time'] = pd.to_datetime(df_oil_price['time'], format='%Y년%m월%d일')
#df_oil_price = df_oil_price.drop(df.index[list(df_WTI[df_WTI.price=='.'].index)])
#df_WTI['price']= df_WTI.price.astype('float')
#df=df.merge(df_oil,how='outer',on="time")

#### 데이터 중 이상치가 있으면 전일 값, 전전일 값으로 대체

In [23]:
df_oil_price.dubai = np.where(df_oil_price.dubai == '-', df_oil_price.dubai.shift(-1) , df_oil_price.dubai)
df_oil_price.dubai = np.where(df_oil_price.dubai == '-', df_oil_price.dubai.shift(-2) , df_oil_price.dubai)
df_oil_price.brent = np.where(df_oil_price.brent == '-', df_oil_price.brent.shift(-1) , df_oil_price.brent)
df_oil_price.brent = np.where(df_oil_price.brent == '-', df_oil_price.brent.shift(-2) , df_oil_price.brent)
df_oil_price.WIT = np.where(df_oil_price.WIT == '-', df_oil_price.WIT.shift(-1) , df_oil_price.WIT)
df_oil_price.WIT = np.where(df_oil_price.WIT == '-', df_oil_price.WIT.shift(-2) , df_oil_price.WIT)

In [24]:
df_oil_price.dubai = df_oil_price.dubai.astype(float)
df_oil_price.brent = df_oil_price.brent.astype(float)
df_oil_price.WIT = df_oil_price.WIT.astype(float)

In [25]:
#기술 통계
df_oil_price.describe()

,dubai,brent,WIT
count,2840.000000,2840.000000,2840.000000
mean,77.314197,80.033173,73.468486
std,25.929948,26.372873,23.115252
min,22.830000,27.880000,26.210000
25%,54.167500,56.007500,51.697500
50%,73.750000,75.585000,72.480000
75%,104.472500,107.737500,93.825000
max,140.700000,146.080000,145.290000


In [26]:
trace_brent = go.Scatter(
                x=df_oil_price.time,
                y=df_oil_price['brent'],
                name = "brent",
                line = dict(color = '#89d5c9'),
                opacity = 0.8)

trace_dubai = go.Scatter(
                x=df_oil_price.time,
                y=df_oil_price['dubai'],
                name = "dubai",
                line = dict(color = '#e25b45'),
                opacity = 0.8)

trace_WIT = go.Scatter(
                x=df_oil_price.time,
                y=df_oil_price['WIT'],
                name = "WIT",
                line = dict(color = '#adc965'),
                opacity = 0.8)
data = [trace_brent,trace_dubai,trace_WIT]
layout = dict(
    title = "국제 유가 추이",)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = "국제 유가 추이")

/usr/local/lib/python3.5/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## News Data Load(국제유가 관련 뉴스)

In [51]:
df = pd.read_csv('data/NewsMainOutput.csv', encoding='UTF-8',sep='|')

In [52]:
df['main']= df.main.str.replace("\t","")
df['main']= df['main'].str.replace('[','')
df['main']= df['main'].str.replace(']','')
df['main']= df['main'].str.replace('동아일보','')
df['main']= df['main'].str.replace('동아닷컴','')
df.dropna(how='any',inplace=True) 
#df['main']= df['main'].astype('str')
data = {'time': df.time.unique(),'main' : df.groupby(['time'])['main'].apply(lambda x: "%s" % ', '.join(x)).values}
df = pd.DataFrame(data=data)

In [53]:
from konlpy.tag import Twitter
tagger = Twitter()

def kor_noun(text):
    words = []
    for w in tagger.nouns(text):
        if len(w) > 1:
            words.append(w)
    return words

/usr/local/lib/python3.5/dist-packages/konlpy/tag/_okt.py:16: UserWarning:

"Twitter" has changed to "Okt" since KoNLPy v0.4.5.



In [65]:
from sklearn.feature_extraction.text import CountVectorizer
from gensim.matutils import Sparse2Corpus
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
cv = CountVectorizer(tokenizer=kor_noun, max_features=1000)
articles = df['main']
tdm = cv.fit_transform(articles)
words = cv.get_feature_names()
corpus = Sparse2Corpus(tdm.T)

dictionary = Dictionary.from_corpus(
    corpus,
    id2word = dict(enumerate(words))
)

lda = LdaModel(corpus=corpus, id2word=dictionary,
               num_topics=1, passes=10, iterations=50,
               random_state=1234)

In [64]:
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda, corpus, dictionary)

/usr/local/lib/python3.5/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




AssertionError: 

#### 기사 당 주제어 10개

In [ ]:
topic_lists[:10]

In [ ]:
df = pd.concat([df,pd.DataFrame(np.array(topic_lists),columns=['top1','top2','top3','top4','top5','top6','top7','top8','top9','top10'])],axis=1)
df = pd.concat([df[['time']],pd.concat([pd.get_dummies(df.top1),pd.get_dummies(df.top2),pd.get_dummies(df.top3)
          ,pd.get_dummies(df.top4),pd.get_dummies(df.top5),pd.get_dummies(df.top6)
          ,pd.get_dummies(df.top7),pd.get_dummies(df.top8),pd.get_dummies(df.top9)
          ,pd.get_dummies(df.top10)], axis=1, sort=False).fillna(0)],axis=1)
df = df.sum(axis=1, level=0)
df['time'] = pd.to_datetime(data['time'], format='%Y.%m.%d.')

In [ ]:
labels =df.columns

In [ ]:
df_WTI = pd.read_csv('DCOILWTICO.csv', encoding='UTF-8')
df_WTI.columns = ['time','price']
df_WTI = df_WTI.drop(df.index[list(df_WTI[df_WTI.price=='.'].index)])
df_WTI['price']= df_WTI.price.astype('float')
df=df.merge(df_oil,how='outer',on="time")

## Load 주유소 기름 Data

In [ ]:
df_oil = pd.read_csv('주유소_평균판매가격_제품별.csv',encoding='UTF-8')
df_oil['time'] = pd.to_datetime(df_oil['구분'], format='%Y년%m월%d일')
df = df_oil[['time','고급휘발유','보통휘발유','자동차용경유']].merge(df,on='time',how='outer').fillna(0)

## 다우존스 / 코스피 Data

In [ ]:
df_dji = pd.read_csv('다우존스 내역.csv', encoding='UTF-8')
kospi = pd.read_csv('코스피지수 내역.csv', encoding='UTF-8')
df_dji['날짜'] = pd.to_datetime(df_dji['날짜'], format='%Y년 %m월 %d일')
kospi['날짜'] = pd.to_datetime(kospi['날짜'], format='%Y년 %m월 %d일')
df_dji['현재가'] = df_dji['현재가'].str.replace(',','').astype('float')
kospi['현재가'] = kospi['현재가'].str.replace(',','').astype('float')
df_stock = df_dji[['날짜','현재가']].merge(kospi[['날짜','현재가']],on='날짜',how='outer')
df_stock['현재가_x'] = df_stock['현재가_x'].fillna(df_stock.현재가_x.astype('float32').mean())
df_stock['현재가_y'] = df_stock['현재가_y'].fillna(df_stock.현재가_y.astype('float32').mean())
df_stock['time'] = df_stock['날짜'].dt.strftime('%Y-%m-%d')
df['time'] = df['time'].dt.strftime('%Y-%m-%d')
df = df.merge(df_stock[['time','현재가_x','현재가_y']],how='outer',on="time")
df['현재가_x'] = df['현재가_x'].fillna(df.현재가_x.astype('float32').mean())
df['현재가_y'] = df['현재가_y'].fillna(df.현재가_y.astype('float32').mean())

## 환율 Data(USD)

In [ ]:
usd_list = []
for date in list(df.time.unique()):
    url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey=E24WPCtlRO2NyplVreVjosPe4XmFFtmb&searchdate={}&data=AP01'.format(date)
    print('{} search start'.format(date))
    try:
        response = requests.get(url)
    except:
        print('retry')
        response = requests.get(url)
    if len(response.json()) == 0:
        usd_list.append(None)
        print('Null')
    else:
        for i in response.json():
            if i['cur_unit'] == 'USD':
                usd_list.append(i['ttb'])
                print(i['ttb'])
    print('{} search end'.format(date))

In [ ]:
pd.DataFrame(data={'er':usd_list,'time':df.time.unique()}).to_csv('er.csv',encoding='UTF-8')

In [ ]:
df_er = pd.read_csv('er.csv',encoding='UTF-8',index_col=0)
df_er['er'] = df_er['er'].str.replace(',','').astype('float')
df_er['er'] = df_er['er'].fillna(df_er['er'].mean())
df_er['time'] = pd.to_datetime(df_er['time'], format='%Y-%m-%d').astype('str')
df = df.merge(df_er,on='time',how='outer')

In [ ]:
df

In [ ]:
!pip install bs4
import urllib
import time
 
from urllib.request import urlopen
from bs4 import BeautifulSoup

total_lists =[]

stockItems = ['096770','010950','011780','020560','003490','004870']

for stockItem in stockItems:
    timestamp = []
    data = []
    print('{} start'.format(stockItem))
    url = 'http://finance.naver.com/item/sise_day.nhn?code='+ stockItem
    html = urlopen(url) 
    source = BeautifulSoup(html.read(), "html.parser")
    maxPage=source.find_all("table",align="center")
    mp = maxPage[0].find_all("td",class_="pgRR")
    #mpNum = int(mp[0].a.get('href')[-3:])
    mpNum = 283
                                         
    for page in range(1, mpNum+1):
        url = 'http://finance.naver.com/item/sise_day.nhn?code=' + stockItem +'&page='+ str(page)
        html = urlopen(url)
        source = BeautifulSoup(html.read(), "html.parser")
        srlists=source.find_all("tr")
        isCheckNone = None
        if((page % 1) == 0):
            time.sleep(1.50)
    for i in range(1,len(srlists)-1):
        if(srlists[i].span != isCheckNone):
            srlists[i].td.text
            timestamp.append(srlists[i].find_all("td",align="center")[0].text)
            data.append(srlists[i].find_all("td",class_="num")[0].text )
    df = pd.DataFrame(data={'time':timestamp,'S-OIL':data})
    total_lists.append(df)

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
096770 start
010950 start


In [ ]:
df[labels] = df[labels].fillna(0)
df[labels].isnull().sum().sum()

In [ ]:
for i in df.drop(labels,axis=1).columns:
    df[i] = df[i].fillna(df[i].mean())
df.isnull().sum().sum()

In [ ]:
df.shape

In [ ]:
df_copy = df.copy()

In [ ]:
df_copy.columns

In [ ]:
df = df.dropna(how='any').set_index('time')

In [ ]:
from keras.models import Sequential, load_model
from keras import layers
from keras import backend as K
from keras import optimizers
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.preprocessing import image

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from datetime import datetime

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalerX = MinMaxScaler()
x = scalerX.fit_transform(df)
scalery = MinMaxScaler()
y = scalery.fit_transform(df.iloc[:,[-1]])

In [ ]:
plt.plot(y)
plt.xlabel("Time Period")
plt.ylabel("Close")

In [ ]:
# Your answer comes here.
SEQ_LEN = 5

# build a dataset
dataX = []
dataY = []

for i in range(0, len(y) - SEQ_LEN):
    _x = x[i:i + SEQ_LEN]
    _y = y[i + SEQ_LEN]
    dataX.append(_x)
    dataY.append(_y)

dataX = np.array(dataX)
dataY = np.array(dataY)

In [ ]:
dataY.shape

In [ ]:
train_size = int(len(dataY) * 0.8)
test_size = len(dataY) - train_size
Xtrain, Xtest = np.array(dataX[0:train_size]), np.array(dataX[train_size:len(dataX)])
ytrain, ytest = np.array(dataY[0:train_size]), np.array(dataY[train_size:len(dataY)])

In [ ]:
np.shape(Xtrain), np.shape(Xtest)

In [ ]:
# Your answer comes here.
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras import backend as K
K.clear_session()
model = Sequential()
model.add(LSTM(128, input_shape=(5, 954),return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mae'])
model.summary()

In [ ]:
# Your answer comes here.
from keras.callbacks import TensorBoard
import time
now = time.strftime("%c")
callbacks_list = [
    ModelCheckpoint(filepath='stock'+now+'.h5',monitor='val_loss',save_best_only=True)
]
history=model.fit(Xtrain, ytrain, validation_data=(Xtest,ytest),epochs=20, verbose=1,callbacks=callbacks_list)

In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="validation loss")
plt.legend()
plt.title("Loss")
plt.show()

In [ ]:
model = load_model('stockSat Jun  8 07:24:44 2019.h5')  # best model 불러오기
test_predict = model.predict(Xtest)
test_predict_back = scalery.inverse_transform(test_predict)
ytest_back = scalery.inverse_transform(ytest)

In [ ]:
# Plot predictions
plt.plot(ytest_back, label = 'Y')
plt.plot(test_predict_back, label = 'Y_predicted')
plt.xlabel("Time Period")
plt.ylabel("Stock Price")
plt.legend()
plt.show()

In [ ]:
mean_absolute_error(test_predict_back, ytest_back), np.mean(test_predict_back / ytest_back) - 1

In [ ]:
test_predict = model.predict(Xtrain)
test_predict_back = scalery.inverse_transform(test_predict)
ytest_back = scalery.inverse_transform(ytrain)

In [ ]:
# Plot predictions
plt.plot(ytest_back, label = 'Y')
plt.plot(test_predict_back, label = 'Y_predicted')
plt.xlabel("Time Period")
plt.ylabel("Stock Price")
plt.legend()
plt.show()